In [28]:
import pygrib

# Ouvrir le fichier GRIB
grbs = pygrib.open('/home/egauillard/extreme_events_forecasting/primary_code/_mars-bol-webmars-public-svc-blue-005-2aed8bececc6a1ac358f1e6ed61bcfa6-DE8eu1.grib')

# Lire tous les messages
for grb in grbs:
    print(grb)


data = grb.values

1:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0 hrs (accum):from 201409020000
2:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-6 hrs (accum):from 201409020000
3:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-12 hrs (accum):from 201409020000


4:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-18 hrs (accum):from 201409020000
5:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-24 hrs (accum):from 201409020000
6:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-30 hrs (accum):from 201409020000
7:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-36 hrs (accum):from 201409020000
8:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-42 hrs (accum):from 201409020000
9:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-48 hrs (accum):from 201409020000
10:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-54 hrs (accum):from 201409020000
11:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-60 hrs (accum):from 201409020000
12:Total Precipitation:kg m**-2 (accum):regular_ll:surface:level 0:fcst time 0-66 hrs (accum):from 20140902000

In [29]:
data

array([[39.0625 , 39.0625 , 39.0625 , ..., 39.0625 , 39.0625 , 39.0625 ],
       [30.625  , 30.65625, 30.71875, ..., 30.15625, 30.3125 , 30.46875],
       [26.59375, 26.6875 , 26.78125, ..., 26.75   , 26.6875 , 26.65625],
       ...,
       [ 1.46875,  1.46875,  1.46875, ...,  1.625  ,  1.5625 ,  1.53125],
       [ 1.96875,  1.96875,  2.     , ...,  2.0625 ,  2.03125,  2.     ],
       [ 4.5625 ,  4.5625 ,  4.5625 , ...,  4.5625 ,  4.5625 ,  4.5625 ]])

## Calculate skill of S2S model ECMWF for daily global reforecasts

In [1]:
import climpred
import xarray as xr

/home/egauillard/extreme_events_forecasting/extreme_events_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dates = xr.cftime_range(start="2020-01-02", freq="7D", end="2020-12-31")
var = "tp"
date = dates[0].strftime("%Y%m%d")
date

'20200102'

In [3]:
import climetlab

In [4]:
forecast_climetlab = (
    climetlab.load_dataset(
        "s2s-ai-challenge-test-input",
        origin="ecmwf",
        date=[20200102, 20200206],  # get to initializations/forecast_times
        parameter=var,
        format="netcdf",
    )
    .to_xarray()
    .compute()
)


By downloading data from this dataset, you agree to the terms and conditions defined at https://apps.ecmwf.int/datasets/data/s2s/licence/. If you do not agree with such terms, do not download the data. 


In [5]:
forecast_climetlab.forecast_time
forecast_climetlab.coords

Coordinates:
  * realization    (realization) int64 0 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
  * forecast_time  (forecast_time) datetime64[ns] 2020-01-02 2020-02-06
  * lead_time      (lead_time) timedelta64[ns] 0 days 1 days ... 45 days 46 days
  * latitude       (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude      (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time     (forecast_time, lead_time) datetime64[ns] 2020-01-02 ... 2...

In [13]:
forecast_times = forecast_climetlab.set_coords("valid_time").valid_time.values.flatten()

# takes a while, continue with obs from above
obs_era = (
    climetlab.load_source(
        "cds",
        "reanalysis-era5-single-levels",
        product_type="reanalysis",
        time=["00:00"],
        grid=[1.5, 1.5],
        param="tp",
        date=forecast_times,
    )
    .to_xarray()
    .squeeze(drop=True)
)

2024-08-20 16:35:55,437 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-20 16:35:55,439 WARNING MOVE TO CDS-Beta
2024-08-20 16:35:55,439 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-08-20 16:35:55,499 INFO Request is queued


KeyboardInterrupt: 

In [9]:
forecast_climetlab.coords

Coordinates:
  * realization    (realization) int64 0 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
  * forecast_time  (forecast_time) datetime64[ns] 2020-01-02 2020-02-06
  * lead_time      (lead_time) timedelta64[ns] 0 days 1 days ... 45 days 46 days
  * latitude       (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude      (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time     (forecast_time, lead_time) datetime64[ns] 2020-01-02 ... 2...

In [11]:
obs_era

<xarray.Dataset>
Dimensions:     (time: 82, latitude: 121, longitude: 240)
Coordinates:
  * time        (time) datetime64[ns] 2020-01-02 2020-01-03 ... 2020-03-23
  * latitude    (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude   (longitude) float64 0.0 1.5 3.0 4.5 ... 354.0 355.5 357.0 358.5
    valid_time  (time) datetime64[ns] ...
Data variables:
    t2m         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-08-20T15:57 GRIB to CDM+CF via cfgrib-0.9.1...

In [10]:
fct = (
    climpred.HindcastEnsemble(
        forecast_climetlab.drop_vars("valid_time").isel(lead_time=range(4))
    )
    .add_observations(obs_era)
    .compute()
)

metric_kwargs = dict(
    metric="rmse", comparison="e2o", alignment="same_inits", dim="init"
)

/home/egauillard/extreme_events_forecasting/extreme_events_env/lib/python3.11/site-packages/climpred/checks.py:201: UserWarning: Did not find dimension "init", but renamed dimension forecast_time with CF-complying standard_name "forecast_reference_time" to init.
  warnings.warn(
/home/egauillard/extreme_events_forecasting/extreme_events_env/lib/python3.11/site-packages/climpred/checks.py:201: UserWarning: Did not find dimension "member", but renamed dimension realization with CF-complying standard_name "realization" to member.
  warnings.warn(
/home/egauillard/extreme_events_forecasting/extreme_events_env/lib/python3.11/site-packages/climpred/checks.py:201: UserWarning: Did not find dimension "lead", but renamed dimension lead_time with CF-complying standard_name "forecast_period" to lead.
  warnings.warn(
SHA256 hash of downloaded file: fdde2643b1d8f988e99b413aa7b3634af332f281c919b9bd5072b15ba2cb521d
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the fi

VariableError: Please provide a Dataset/DataArray with at least one matching variable to the initialized prediction ensemble; got Data variables:
    tp       (member, init, lead, latitude, longitude) float32 0.0 ... 0.3789 for init and Data variables:
    t2m      (time, latitude, longitude) float32 ... for verif.

In [ ]:
skill = fct.verify(**metric_kwargs)
skill[var].plot(col="lead", robust=True)

## biweekly aggregate 

In [ ]:
from climpred.utils import convert_Timedelta_to_lead_units

forecast = convert_Timedelta_to_lead_units(
    forecast_climetlab.rename({"lead_time": "lead"})
)


In [ ]:
# create 14D averages
forecast_w12 = forecast.sel(lead=range(1, 14)).mean(dim="lead")
forecast_w34 = forecast.sel(lead=range(14, 28)).mean(dim="lead")
forecast_w56 = forecast.sel(lead=range(28, 42)).mean(dim="lead")
forecast_biweekly = xr.concat([forecast_w12, forecast_w34, forecast_w56], dim="lead")
forecast_biweekly["lead"] = [
    1,
    14,
    28,
]  # lead represents first day of biweekly aggregate
forecast_biweekly["lead"].attrs["units"] = "days"
forecast_biweekly.coords


In [ ]:
# 14D rolling mean
obs_biweekly = obs_era.rolling(time=14, center=False).mean()
obs_biweekly = obs_biweekly.isel(time=slice(13, None)).assign_coords(
    time=obs_era.time.isel(time=slice(None, -13))
)  # time represents first day of the biweekly aggregate

In [ ]:
fct_biweekly = climpred.HindcastEnsemble(forecast_biweekly).add_observations(
    obs_biweekly
)

In [ ]:
obs_edges = (
    obs_biweekly.groupby("time.month")
    .quantile(q=[1 / 3, 2 / 3], dim="time", skipna=False)
    .rename({"quantile": "category_edge"})
)

In [ ]:
model_edges = (
    forecast_biweekly.rename({"forecast_time": "init", "realization": "member"})
    .groupby("init.month")
    .quantile(q=[1 / 3, 2 / 3], dim=["init", "member"], skipna=False)
    .rename({"quantile": "category_edge"})
)

In [ ]:
 # use the same observational category_edges for observations and forecasts
rps_same_edges = fct_biweekly.verify(
    metric="rps",
    comparison="m2o",
    alignment="same_inits",
    dim=["member", "init"],
    category_edges=obs_edges,
)

rps_same_edges.t2m.plot(col="lead", robust=True)

In [ ]:
# use different observational category_edges for observations and forecast category_edges for forecasts
rps_different_edges = fct_biweekly.verify(
    metric="rps",
    comparison="m2o",
    alignment="same_inits",
    dim=["member", "init"],
    category_edges=(obs_edges, model_edges),
)

rps_different_edges.t2m.plot(col="lead", robust=True)

In [ ]:
ds = climetlab.load_dataset(
    "era5-precipitations", period=(2020, 2023), domain="Ireland", time=12
)